In [37]:
using Pkg
#Pkg.add("JuMP")
#Pkg.add("HiGHS")
#Pkg.add("Gurobi")
#Pkg.add(url ="https://github.com/rafaelmartinelli/BPPLib.jl")
#Pkg.add("Random")
#Pkg.add("Statistics")
#Pkg.add("Plots")
#Pkg.add("LinearAlgebra")


PART1


In [38]:
using JuMP
using Gurobi
using HiGHS
using BPPLib
using Random
using Statistics
using Plots

struct Pattern
    x::Vector{Int}
end

function addpattern!(P::Vector{Pattern}, x::Vector{Int})
    if sum(x) == 0
        return
    end
    for p in P
        if p.x == x
            return 
        end
    end
    push!(P, Pattern(copy(x)))
end

function greedy_fill!(r::Vector{Int}, l::Vector{Int}, L::Int)
    n = length(l)
    x = zeros(Int, n)
    used = 0
    while true
        idx = [j for j in 1:n if r[j] > 0 && used + l[j] <= L]
        isempty(idx) && break
        jbest = argmax([(l[j], r[j]) for j in idx])
        j = idx[jbest]
        x[j] += 1
        r[j] -= 1
        used += l[j]
    end
    return x
end

function build_patterns(l::Vector{Int}, d::Vector{Int}, L::Int; m_max::Int=200)
    n = length(l)
    P = Pattern[]

    for j in 1:n
        if l[j] <= L
            x = zeros(Int, n)
            x[j] = 1
            addpattern!(P, x)
        end
    end
    for a in 1:n, b in a:n
        if l[a] + l[b] <= L
            x = zeros(Int, n)
            x[a] += 1
            x[b] += 1
            addpattern!(P, x)
        end
    end
    r = copy(d)
    k = 0
    while k < m_max && sum(r) > 0
        x = greedy_fill!(r, l, L)
        addpattern!(P, x)
        k += 1
    end
    return P
end

function solve_P1(L::Int, l::Vector{Int}, d::Vector{Int}, P::Vector{Pattern};
                  timelimit::Float64=200.0)
    T = length(P)
    model = Model(HiGHS.Optimizer) #Gurobi quand licence ok
    set_silent(model)
    set_attribute(model, "time_limit", timelimit)

    @variable(model, y[1:T] >= 0, Int)
    @objective(model, Min, sum(y[t] for t in 1:T))
    @constraint(model, cover[j=1:length(l)], sum(P[t].x[j]*y[t] for t in 1:T) >= d[j])

    optimize!(model)
    obj = objective_value(model)
    status = termination_status(model)
    time_s = JuMP.MOI.get(model, JuMP.MOI.SolveTimeSec())
    return obj, status, time_s, value.(y)
end

function toy_example()
    L = 5
    l = [2, 2, 3]
    d = [4, 3, 1]
    P = build_patterns(l, d, L; m_max=50)
    obj, status, t, value = solve_P1(L, l, d, P)
    println("  Objective (nbr of long boards used) = ", obj, "   Status = ", status, "   Time = ", round(t, digits=3), " s")
end

toy_example()







                



  Objective (nbr of long boards used) = 4.0   Status = OPTIMAL   Time = 0.015 s


In [39]:
function write_csp_instance(filename::String, name::String, L::Int, l::Vector{Int}, d::Vector{Int})
    open(filename, "w") do io
        println(io, length(l), " ", L)
        for j in 1:length(l)
            println(io, l[j], " ", d[j])
        end
    end
end

function generate_csp_instance(n ::Int=100)
    l = rand(2:30, n)
    d = rand(1:10, n)
    L = Int(round(mean(l) * 4))   #keep patterns feasible
     
    return L, l, d
end

function solve_csp_file(filename::String; m_max::Int=300)
    data = BPPLib.loadCSP(filename)

    L = Int(data.capacity)
    l = Vector{Int}(data.weights)
    d = Vector{Int}(data.demands)
    println("  Capacity = ", L, " | n = ", length(l))

    P = build_patterns(l, d, L; m_max=m_max)
    obj, status, t, _ = solve_P1(L, l, d, P)

    println("  Objective (primal bound): ", obj)
    println("  Status: ", status, " | Time: ", round(t, digits=2), " s")
end


L_big, l_big, d_big = generate_csp_instance()
write_csp_instance("my_instance.txt", "MyInstance", L_big, l_big, d_big)

solve_csp_file("my_instance.txt"; m_max=400)

  Capacity = 67 | n = 100
  Objective (primal bound): 143.0
  Status: OPTIMAL | Time: 0.07 s


In [40]:
"""

sizes = [1,2,5,10,20,30,50,75,100,150,200]
times = Float64[]
for n in sizes
    L, l, d = generate_csp_instance(n)
    write_csp_instance("temp_instance.txt", "TempInstance", L, l, d)
    println("Solving instance of size n = ", n)
    data = BPPLib.loadCSP("temp_instance.txt")
    L = Int(data.capacity)
    l = Vector{Int}(data.weights)
    d = Vector{Int}(data.demands)
    P = build_patterns(l, d, L; m_max=300)
    _, _, t, _ = solve_P1(L, l, d, P)
    push!(times, t)
end
plot(sizes, times, xlabel="Instance Size (n)", ylabel="Time (s)", title="Solving Time", legend=false, marker=:circle)
savefig("csp_solving_time.pdf")"""


"\nsizes = [1,2,5,10,20,30,50,75,100,150,200]\ntimes = Float64[]\nfor n in sizes\n    L, l, d = generate_csp_instance(n)\n    write_csp_instance(\"temp_instance.txt\", \"TempInstance\", L, l, d)\n    println(\"Solving instance of size n = \", n)\n    data = BPPLib.loadCSP(\"temp_instan" ⋯ 87 bytes ⋯ "t}(data.demands)\n    P = build_patterns(l, d, L; m_max=300)\n    _, _, t, _ = solve_P1(L, l, d, P)\n    push!(times, t)\nend\nplot(sizes, times, xlabel=\"Instance Size (n)\", ylabel=\"Time (s)\", title=\"Solving Time\", legend=false, marker=:circle)\nsavefig(\"csp_solving_time.pdf\")"

PART2

In [41]:
using LinearAlgebra

function hungarian_method(C::AbstractMatrix{<:Real})
    m, n = size(C)
    if m > n
        C = hcat(C, zeros(m, m - n))
        n = m
    elseif n > m
        C = vcat(C, zeros(n - m, n))
        m = n
    end
    println("Cost matrix size: ", size(C),C)
    model = Model(HiGHS.Optimizer)
    set_silent(model)

    @variable(model, x[1:m, 1:n], Bin)
    @objective(model, Min, sum(C[i,j] * x[i,j] for i in 1:m, j in 1:n))
    @constraint(model, [i=1:m], sum(x[i,j] for j in 1:n) == 1)
    @constraint(model, [j=1:n], sum(x[i,j] for i in 1:m) == 1)

    optimize!(model)
    assignment = zeros(Int, m)
    for i in 1:m
        for j in 1:n
            if value(x[i,j]) > 0.5
                assignment[i] = j
            end
        end
    end
    return assignment, objective_value(model)
end

C = [4 2 8;
     2 4 6;
     8 6 4]
     
assignment, cost = hungarian_method(C)
println("Optimal assignment: ", assignment)

C2 = [4 2 8 3;
      2 4 6 5;
      8 6 4 7]
assignment2, cost2 = hungarian_method(C2)
println("Optimal assignment for non-square matrix: ", assignment2)


Cost matrix size: (3, 3)[4 2 8; 2 4 6; 8 6 4]
Optimal assignment: [2, 1, 3]
Cost matrix size: (4, 4)[4.0 2.0 8.0 3.0; 2.0 4.0 6.0 5.0; 8.0 6.0 4.0 7.0; 0.0 0.0 0.0 0.0]
Optimal assignment for non-square matrix: [2, 1, 3, 4]
